<a href="https://colab.research.google.com/github/sirinebenyedder/AMERF/blob/main/Serveur.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

uploaded = files.upload()


Saving 000028.jpg to 000028.jpg


In [ ]:
!pip install ultralytics
!pip install -q flask pyngrok ultralytics opencv-python

In [ ]:
!pip install -q flask flask-ngrok pyngrok ultralytics opencv-python

In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("37Czy4vPp8VksVRu38SwaNRhdAB_x3deBEgUPUqh99UzBJz4")


In [ ]:

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import defaultdict, deque
from datetime import datetime, timedelta
import hashlib
import json
from typing import Dict, List, Tuple, Optional, Any, Union
import warnings
from dataclasses import dataclass
from enum import Enum
import time
import cv2
import os
import uuid
import logging
from flask import Flask, request, jsonify
from pyngrok import ngrok
import threading
import atexit

from ultralytics import YOLO


In [ ]:
class AttackType(Enum):
    """Types d'attaques détectées"""
    NORMAL = "normal"
    EXTRACTION = "extraction"
    CLONING = "cloning"
    INVERSION = "inversion"
    MEMBERSHIP_INFERENCE = "membership_inference"


In [ ]:
@dataclass
class QueryStats:
    user_id: str
    total_queries: int = 0
    suspicious_queries: int = 0
    risk_score: float = 0.0
    last_activity: datetime = None
    attack_type: AttackType = AttackType.NORMAL

    def to_dict(self):
        return {
            'user_id': self.user_id,
            'total_queries': self.total_queries,
            'suspicious_queries': self.suspicious_queries,
            'risk_score': round(self.risk_score, 3),
            'last_activity': self.last_activity.isoformat() if self.last_activity else None,
            'attack_type': self.attack_type.value
        }


class YOLOQueryPatternFingerprinting:

    def __init__(self,
                 window_size: int = 50,
                 time_window: int = 60,
                 risk_threshold: float = 0.7,
                 history_file: str = "global_history.json"):

        self.window_size = window_size
        self.time_window = time_window
        self.risk_threshold = risk_threshold
        self.history_file = history_file

        # Charger l'historique global depuis JSON
        self.global_history = self._load_global_history()

        # Cache mémoire
        self.query_cache = defaultdict(lambda: {
            'timestamps': deque(maxlen=window_size),
            'image_hashes': deque(maxlen=window_size),
            'image_shapes': deque(maxlen=window_size)
        })

    def _load_global_history(self) -> Dict:
        try:
            if os.path.exists(self.history_file):
                with open(self.history_file, 'r') as f:
                    history = json.load(f)

                    # Convertir les timestamps en datetime de manière robuste
                    for user_id, user_data in history.get('users', {}).items():
                        if 'timestamps' in user_data and user_data['timestamps']:
                            timestamps = []
                            for ts in user_data['timestamps']:
                                if isinstance(ts, str):
                                    try:
                                        # Convertir ISO string en datetime
                                        if 'T' in ts:
                                            # Format ISO avec 'T'
                                            dt = datetime.fromisoformat(ts)
                                        else:
                                            # Format plus simple
                                            dt = datetime.strptime(ts, "%Y-%m-%d %H:%M:%S.%f")
                                        timestamps.append(dt)
                                    except Exception as e:
                                        print(f"⚠️  Erreur conversion timestamp {ts}: {e}")
                                        timestamps.append(datetime.now())
                                else:
                                    timestamps.append(ts)
                            user_data['timestamps'] = timestamps

                        # Convertir first_seen et last_seen
                        for field in ['first_seen', 'last_seen']:
                            if field in user_data and user_data[field]:
                                if isinstance(user_data[field], str):
                                    try:
                                        user_data[field] = datetime.fromisoformat(user_data[field])
                                    except:
                                        user_data[field] = datetime.now()

                    print(f"Historique chargé: {history.get('global_stats', {}).get('total_queries', 0)} requêtes")
                    return history
            else:
                print("ℹ️  Création nouveau fichier d'historique")
                return {
                    'users': {},
                    'global_stats': {
                        'total_queries': 0,
                        'suspicious_queries': 0,
                        'unique_users': 0,
                        'start_time': datetime.now().isoformat()
                    },
                    'last_updated': datetime.now().isoformat()
                }
        except Exception as e:
            print(f" Erreur chargement historique: {e}")
            return {
                'users': {},
                'global_stats': {
                    'total_queries': 0,
                    'suspicious_queries': 0,
                    'unique_users': 0,
                    'start_time': datetime.now().isoformat()
                }
            }

    def _save_global_history(self):
        try:
            history_copy = self.global_history.copy()

            for user_id, user_data in history_copy.get('users', {}).items():
                if 'timestamps' in user_data and user_data['timestamps']:
                    # Convertir les datetime en strings ISO
                    user_data['timestamps'] = [
                        ts.isoformat() if isinstance(ts, datetime) else str(ts)
                        for ts in user_data['timestamps']
                    ]
                # Convertir first_seen et last_seen
                for field in ['first_seen', 'last_seen']:
                    if field in user_data and user_data[field]:
                        if isinstance(user_data[field], datetime):
                            user_data[field] = user_data[field].isoformat()
                        elif isinstance(user_data[field], str):
                            pass

            history_copy['last_updated'] = datetime.now().isoformat()

            with open(self.history_file, 'w') as f:
                json.dump(history_copy, f, indent=2, default=str)

        except Exception as e:
            print(f"Erreur sauvegarde historique: {e}")

    def compute_image_hash(self, image: np.ndarray) -> str:
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image

        resized = cv2.resize(gray, (64, 64))
        hash_str = hashlib.sha256(resized.tobytes()).hexdigest()
        return hash_str

    def update_global_history(self, user_id: str, image_hash: str,
                            risk_score: float, is_suspicious: bool,
                            attack_type: AttackType):
        """Met à jour l'historique global - CORRIGÉ"""
        current_time = datetime.now()

        if user_id not in self.global_history['users']:
            self.global_history['users'][user_id] = {
                'total_queries': 0,
                'suspicious_queries': 0,
                'risk_scores': [],
                'image_hashes': [],
                'timestamps': [],
                'attack_types': defaultdict(int),
                'first_seen': current_time,
                'last_seen': current_time
            }

        user_data = self.global_history['users'][user_id]

        # Mettre à jour les données utilisateur
        user_data['total_queries'] += 1
        user_data['last_seen'] = current_time

        if len(user_data['risk_scores']) >= 50:
            user_data['risk_scores'].pop(0)
            user_data['image_hashes'].pop(0)
            if user_data['timestamps']:
                user_data['timestamps'].pop(0)

        user_data['risk_scores'].append(risk_score)
        user_data['image_hashes'].append(image_hash[:16])
        user_data['timestamps'].append(current_time)

        if is_suspicious:
            user_data['suspicious_queries'] += 1

        # Mettre à jour les types d'attaque
        attack_str = attack_type.value
        user_data['attack_types'][attack_str] = user_data['attack_types'].get(attack_str, 0) + 1

        # Mettre à jour les statistiques globales
        self.global_history['global_stats']['total_queries'] += 1
        if is_suspicious:
            self.global_history['global_stats']['suspicious_queries'] += 1

        self.global_history['global_stats']['unique_users'] = len(self.global_history['users'])

        # Sauvegarder périodiquement
        if self.global_history['global_stats']['total_queries'] % 5 == 0:
            self._save_global_history()

    def analyze_query_pattern(self, user_id: str, image: np.ndarray) -> Dict[str, Any]:
        """Analyse le pattern de requête avec historique global - VERSION CORRIGÉE"""
        current_time = datetime.now()

        image_hash = self.compute_image_hash(image)
        image_shape = image.shape

        cache_entry = self.query_cache[user_id]
        cache_entry['timestamps'].append(current_time)
        cache_entry['image_hashes'].append(image_hash)
        cache_entry['image_shapes'].append(image_shape)

        user_history = self.global_history['users'].get(user_id, {})

        if len(cache_entry['timestamps']) < 2 and user_history.get('total_queries', 0) == 0:
            return {
                'risk_score': 0.0,
                'is_suspicious': False,
                'attack_type': AttackType.NORMAL,
                'metrics': {
                    'frequency': 0.0,
                    'diversity': 1.0,
                    'similarity': 0.0,
                    'repeat_score': 0.0
                },
                'image_hash': image_hash[:16]
            }

        #  Gestion des timestamps
        all_timestamps = []
        if user_history.get('timestamps'):
            for ts in user_history['timestamps'][-20:]:  # Dernières 20
                if isinstance(ts, datetime):
                    all_timestamps.append(ts)
                elif isinstance(ts, str):
                    try:
                        all_timestamps.append(datetime.fromisoformat(ts))
                    except:
                        all_timestamps.append(current_time)

        all_timestamps.extend(cache_entry['timestamps'])

        # 1. Fréquence des requêtes
        if len(all_timestamps) >= 2:
            time_diffs = []
            for i in range(1, min(10, len(all_timestamps))):
                try:
                    diff = (all_timestamps[i] - all_timestamps[i-1]).total_seconds()
                    time_diffs.append(diff)
                except:
                    continue

            if time_diffs:
                avg_frequency = np.mean(time_diffs)
                frequency_score = min(1.0, 5.0 / (avg_frequency + 0.1))
            else:
                avg_frequency = 60
                frequency_score = 0.0
        else:
            avg_frequency = 60
            frequency_score = 0.0

        # 2. Diversité des images
        all_hashes = []
        if user_history.get('image_hashes'):
            all_hashes.extend(user_history['image_hashes'][-20:])
        all_hashes.extend([h[:16] for h in cache_entry['image_hashes']])

        unique_hashes = len(set(all_hashes))
        total_queries = len(all_hashes)
        diversity = unique_hashes / max(total_queries, 1)
        diversity_score = 1.0 - diversity

        # 3. Score de répétition
        current_hash_short = image_hash[:16]
        repeat_count = all_hashes.count(current_hash_short)
        repeat_score = min(0.9, repeat_count * 0.3)

        # 4. Similarité entre images consécutives
        if len(cache_entry['image_hashes']) >= 2:
            similarities = []
            for i in range(1, min(5, len(cache_entry['image_hashes']))):
                hash1 = cache_entry['image_hashes'][-i]
                hash2 = cache_entry['image_hashes'][-i-1]
                similarity = 1.0 if hash1[:8] == hash2[:8] else 0.0
                similarities.append(similarity)

            avg_similarity = np.mean(similarities) if similarities else 0.0
            pattern_score = avg_similarity if avg_similarity > 0.8 else 0.0
        else:
            avg_similarity = 0.0
            pattern_score = 0.0

        # 5. Score de risque
        risk_score = (
            0.4 * frequency_score +      # Fréquence
            0.3 * repeat_score +         # Répétition (nouveau)
            0.2 * diversity_score +      # Diversité
            0.1 * pattern_score          # Pattern
        )

        # 6. Ajuster basé sur l'historique de suspicion
        if user_history and user_history.get('total_queries', 0) > 0:
            historical_suspicious_rate = user_history.get('suspicious_queries', 0) / user_history['total_queries']
            risk_score = min(1.0, risk_score * (1 + historical_suspicious_rate))

        # 7. Détection du type d'attaque
        attack_type = AttackType.NORMAL
        is_suspicious = risk_score > self.risk_threshold

        if is_suspicious:
            if repeat_score > 0.6:  # Forte répétition = CLONING
                attack_type = AttackType.CLONING
            elif frequency_score > 0.7 and diversity_score < 0.4:
                attack_type = AttackType.EXTRACTION
            elif diversity_score > 0.6 and frequency_score < 0.3:
                attack_type = AttackType.MEMBERSHIP_INFERENCE

        # 8. Métriques détaillées
        metrics = {
            'frequency_seconds': round(avg_frequency, 3) if 'avg_frequency' in locals() else 0.0,
            'frequency_score': round(frequency_score, 3),
            'diversity_ratio': round(diversity, 3),
            'diversity_score': round(diversity_score, 3),
            'repeat_count': repeat_count,
            'repeat_score': round(repeat_score, 3),
            'similarity': round(avg_similarity, 3),
            'pattern_score': round(pattern_score, 3),
            'total_queries': total_queries,
            'unique_images': unique_hashes,
            'image_hash': image_hash[:16]
        }

        return {
            'risk_score': round(float(risk_score), 3),
            'is_suspicious': is_suspicious,
            'attack_type': attack_type,
            'metrics': metrics
        }

    def get_user_statistics(self, user_id: str) -> Optional[Dict]:
        """Obtient les statistiques d'un utilisateur"""
        if user_id not in self.global_history['users']:
            return None

        user_data = self.global_history['users'][user_id]

        # Calculer le temps total
        if user_data.get('timestamps') and len(user_data['timestamps']) > 1:
            first = user_data['timestamps'][0]
            last = user_data['timestamps'][-1]
            if isinstance(first, datetime) and isinstance(last, datetime):
                time_span = (last - first).total_seconds()
            else:
                time_span = 0
        else:
            time_span = 0

        return {
            'total_queries': user_data.get('total_queries', 0),
            'suspicious_queries': user_data.get('suspicious_queries', 0),
            'suspicious_rate': round(user_data.get('suspicious_queries', 0) / max(user_data.get('total_queries', 1), 1), 3),
            'avg_risk_score': round(np.mean(user_data.get('risk_scores', [])) if user_data.get('risk_scores') else 0, 3),
            'max_risk_score': round(max(user_data.get('risk_scores', [0])) if user_data.get('risk_scores') else 0, 3),
            'unique_images': len(set(user_data.get('image_hashes', []))),
            'time_span_seconds': round(time_span, 1),
            'first_seen': user_data.get('first_seen', datetime.now()).isoformat() if isinstance(user_data.get('first_seen'), datetime) else user_data.get('first_seen'),
            'last_seen': user_data.get('last_seen', datetime.now()).isoformat() if isinstance(user_data.get('last_seen'), datetime) else user_data.get('last_seen'),
            'attack_distribution': dict(user_data.get('attack_types', {}))
        }

    def get_global_statistics(self) -> Dict:
        """Retourne les statistiques globales"""
        stats = self.global_history['global_stats']

        suspicion_rate = stats['suspicious_queries'] / max(stats['total_queries'], 1)

        attack_distribution = defaultdict(int)
        for user_data in self.global_history['users'].values():
            for attack_type, count in user_data.get('attack_types', {}).items():
                attack_distribution[attack_type] += count

        return {
            'total_queries': stats['total_queries'],
            'suspicious_queries': stats['suspicious_queries'],
            'suspicion_rate': round(suspicion_rate * 100, 1),
            'unique_users': stats['unique_users'],
            'attack_distribution': dict(attack_distribution),
            'start_time': stats.get('start_time'),
            'last_updated': self.global_history.get('last_updated')
        }


class YOLODynamicPredictionNoise:
    """DPN"""

    def __init__(self, base_noise: float = 0.02, max_noise: float = 0.25):
        self.base_noise = base_noise
        self.max_noise = max_noise
        self.noise_history = defaultdict(list)

    def calculate_adaptive_noise(self, risk_score: float) -> float:
        noise_level = self.base_noise + (self.max_noise - self.base_noise) * (risk_score ** 1.5)
        return min(noise_level, 0.5)

    def apply_noise_to_results(self, results, risk_score: float, user_id: str = "anonymous"):
        if risk_score < 0.2:
            return results, {
                'noise_level': 0.0,
                'original_confidences': [],
                'noisy_confidences': [],
                'avg_confidence_change': 0.0,
                'max_confidence_change': 0.0
            }

        noise_level = self.calculate_adaptive_noise(risk_score)

        original_confidences = []
        noisy_confidences = []
        noise_applied_count = 0

        for result in results:
            if hasattr(result, 'boxes') and result.boxes is not None:
                try:
                    boxes_data = result.boxes.data
                    if boxes_data is not None and len(boxes_data) > 0:
                        boxes_data_clone = boxes_data.clone()

                        if boxes_data_clone.shape[1] >= 5:
                            conf_tensor = boxes_data_clone[:, 4]

                            # Store original confidences
                            for conf in conf_tensor:
                                original_confidences.append(conf.item())

                            # Apply noise
                            noise_conf = torch.randn_like(conf_tensor) * noise_level * 0.3
                            noisy_conf = torch.clamp(conf_tensor + noise_conf, 0.01, 0.99)

                            # Store noisy confidences
                            for conf in noisy_conf:
                                noisy_confidences.append(conf.item())

                            boxes_data_clone[:, 4] = noisy_conf
                            result.boxes.data = boxes_data_clone

                            noise_applied_count += len(conf_tensor)

                except Exception as e:
                    print(f" Erreur DPN: {e}")
                    continue

        # Calculate confidence
        confidence_changes = []
        if original_confidences and noisy_confidences:
            for orig, noisy in zip(original_confidences, noisy_confidences):
                confidence_changes.append(abs(orig - noisy))

        avg_confidence_change = np.mean(confidence_changes) if confidence_changes else 0.0
        max_confidence_change = max(confidence_changes) if confidence_changes else 0.0

        # Enregistrer l'application du bruit
        self.noise_history[user_id].append({
            'timestamp': datetime.now(),
            'risk_score': risk_score,
            'noise_level': noise_level,
            'avg_confidence_change': avg_confidence_change,
            'max_confidence_change': max_confidence_change,
            'boxes_affected': noise_applied_count
        })

        return results, {
            'noise_level': noise_level,
            'original_confidences': original_confidences,
            'noisy_confidences': noisy_confidences,
            'avg_confidence_change': avg_confidence_change,
            'max_confidence_change': max_confidence_change,
            'boxes_affected': noise_applied_count
        }


class ProtectedYOLO:
    """
     historique global JSON
    """

    def __init__(self,
                 model_path: str,
                 watermark_key: str = "yolo_protection_key",
                 config: Dict[str, Any] = None):

        # Charger le modèle YOLO
        print(f" Chargement du modèle YOLO depuis {model_path}...")
        self.base_yolo = YOLO(model_path)
        self.model = self.base_yolo.model

        # Configuration
        default_config = {
            'qpf_window_size': 5,
            'risk_threshold': 0.5,
            'dpn_max_noise': 0.15,
            'verbose': True
        }
        if config:
            default_config.update(config)
        self.config = default_config

        # Initialiser QPF avec historique global
        self.qpf = YOLOQueryPatternFingerprinting(
            window_size=self.config['qpf_window_size'],
            risk_threshold=self.config['risk_threshold'],
            history_file="amerf_global_history.json"
        )

        # Initialiser DPN
        self.dpn = YOLODynamicPredictionNoise(
            max_noise=self.config['dpn_max_noise']
        )

        # Statistiques globales
        self.global_stats = {
            'total_predictions': 0,
            'suspicious_predictions': 0,
            'last_log_time': datetime.now()
        }

        # Enregistrer pour sauvegarde à la fermeture
        atexit.register(self._cleanup)
        print("Modèle YOLO protégé ")

    def _cleanup(self):
        """Sauvegarde l'historique à la fermeture"""
        self.qpf._save_global_history()
        print(" Historique global sauvegardé")

    def _load_image(self, source: Union[str, np.ndarray]) -> np.ndarray:
        if isinstance(source, str):
            image = cv2.imread(source)
            if image is None:
                raise ValueError(f"Impossible de charger l'image: {source}")
            return image
        elif isinstance(source, np.ndarray):
            return source
        else:
            try:
                return np.array(source)
            except:
                raise ValueError(f"Type de source non supporté: {type(source)}")

    def predict(self,
                source: Union[str, np.ndarray],
                user_id: str = "anonymous",
                return_debug: bool = False,
                **kwargs):

        start_time = time.time()

        try:
            # Charger l'image
            image = self._load_image(source)

            # 1. ANALYSE COMPORTEMENTALE
            fingerprint = self.qpf.analyze_query_pattern(user_id, image)

            risk_score = fingerprint['risk_score']
            is_suspicious = fingerprint['is_suspicious']
            attack_type = fingerprint['attack_type']
            metrics = fingerprint['metrics']

            # Mettre à jour les stats
            self.global_stats['total_predictions'] += 1
            if is_suspicious:
                self.global_stats['suspicious_predictions'] += 1

            # 2. Mettre à jour l'historique global
            image_hash = self.qpf.compute_image_hash(image)
            self.qpf.update_global_history(user_id, image_hash, risk_score, is_suspicious, attack_type)

            # 3. EXÉCUTER L'INFÉRENCE
            results = self.base_yolo(image, **kwargs)

            # 4. ANALYSER LA PRÉCISION RÉELLE AVANT BRUIT
            real_precision_metrics = self._analyze_real_precision(results)

            # 5. APPLIQUER LE BRUIT SI NÉCESSAIRE
            noise_applied = False
            noise_info = None
            if is_suspicious:
                results, noise_info = self.dpn.apply_noise_to_results(results, risk_score, user_id)
                noise_applied = True

            # 6. ANALYSER LA PRÉCISION APRÈS BRUIT
            noisy_precision_metrics = self._analyze_noisy_precision(results, noise_info)

            execution_time = time.time() - start_time

            # 7. LOGS EN TEMPS RÉEL
            current_time = datetime.now()
            time_since_last_log = (current_time - self.global_stats['last_log_time']).total_seconds()

            if time_since_last_log > 2:  # Log toutes les 2 secondes max
                status = "🚨 SUSPECT" if is_suspicious else " NORMAL"
                print(f"\n{'='*60}")
                print(f" User: {user_id}")
                print(f" Score risque: {risk_score:.3f} | Type: {attack_type.value}")
                print(f"Métriques:")
                print(f"  • Fréquence: {metrics['frequency_seconds']:.1f}s (score: {metrics['frequency_score']:.2f})")
                print(f"  • Diversité: {metrics['diversity_ratio']:.1%} (score: {metrics['diversity_score']:.2f})")
                print(f"  • Répétition: {metrics['repeat_count']}x (score: {metrics['repeat_score']:.2f})")
                print(f"  • Images uniques: {metrics['unique_images']}/{metrics['total_queries']}")

                # PRÉCISION AVANT/APRÈS BRUIT
                print(f" PRÉCISION DES CONFIDENCES:")
                print(f"  • Réelle (avant bruit):")
                print(f"     - Moyenne: {real_precision_metrics['avg_confidence']:.3f}")
                print(f"     - Max: {real_precision_metrics['max_confidence']:.3f}")
                print(f"     - Min: {real_precision_metrics['min_confidence']:.3f}")
                print(f"     - Détections: {real_precision_metrics['total_boxes']}")

                if noise_applied and noise_info:
                    print(f"  • Après bruit (niveau: {noise_info['noise_level']:.3f}):")
                    print(f"     - Moyenne: {noisy_precision_metrics['avg_confidence']:.3f}")
                    print(f"     - Max: {noisy_precision_metrics['max_confidence']:.3f}")
                    print(f"     - Min: {noisy_precision_metrics['min_confidence']:.3f}")
                    print(f"     - Δ Moyenne: {noise_info['avg_confidence_change']:.3f}")
                    print(f"     - Δ Max: {noise_info['max_confidence_change']:.3f}")
                    print(f"     - Boîtes affectées: {noise_info['boxes_affected']}")
                else:
                    print(f"  • Après bruit: AUCUN (utilisateur normal)")

                print(f" Défenses: Bruit appliqué: {noise_applied}")
                print(f" Performance: {execution_time:.2f}s")
                print(f"{'='*60}")
                self.global_stats['last_log_time'] = current_time

            if return_debug:
                debug_info = {
                    'fingerprint': fingerprint,
                    'execution_time': execution_time,
                    'user_stats': self.qpf.get_user_statistics(user_id),
                    'global_stats': self.qpf.get_global_statistics(),
                    'image_hash': image_hash[:16],
                    'noise_applied': noise_applied,
                    'noise_info': noise_info,
                    'real_precision': real_precision_metrics,
                    'noisy_precision': noisy_precision_metrics
                }
                return results, debug_info

            return results

        except Exception as e:
            print(f" Erreur dans predict: {e}")
            if return_debug:
                return None, {'error': str(e)}
            return None

    def _analyze_real_precision(self, results):
        """Analyse la précision réelle avant l'application du bruit"""
        all_confidences = []
        total_boxes = 0

        if not results:
            return {
                'avg_confidence': 0.0,
                'max_confidence': 0.0,
                'min_confidence': 0.0,
                'total_boxes': 0
            }

        if isinstance(results, list):
            for result in results:
                if hasattr(result, 'boxes') and result.boxes is not None:
                    boxes_data = result.boxes.data
                    if boxes_data is not None and len(boxes_data) > 0:
                        if boxes_data.shape[1] >= 5:
                            confidences = boxes_data[:, 4].tolist()
                            all_confidences.extend(confidences)
                            total_boxes += len(confidences)

        if all_confidences:
            return {
                'avg_confidence': np.mean(all_confidences),
                'max_confidence': max(all_confidences),
                'min_confidence': min(all_confidences),
                'total_boxes': total_boxes
            }
        else:
            return {
                'avg_confidence': 0.0,
                'max_confidence': 0.0,
                'min_confidence': 0.0,
                'total_boxes': 0
            }

    def _analyze_noisy_precision(self, results, noise_info):
        """Analyse la précision après l'application du bruit"""
        if noise_info and noise_info['noisy_confidences']:
            noisy_confidences = noise_info['noisy_confidences']
            return {
                'avg_confidence': np.mean(noisy_confidences),
                'max_confidence': max(noisy_confidences),
                'min_confidence': min(noisy_confidences),
                'total_boxes': len(noisy_confidences)
            }
        else:
            # Si pas de bruit, retourner les mêmes métriques que réelles
            return self._analyze_real_precision(results)

    def get_protection_stats(self, user_id: str = None) -> Dict[str, Any]:
        """statistiques de protection"""
        if user_id:
            user_stats = self.qpf.get_user_statistics(user_id)
            return {
                'user_stats': user_stats,
                'global_stats': self.qpf.get_global_statistics()
            }
        else:
            return {
                'global_stats': self.qpf.get_global_statistics(),
                'model_stats': self.global_stats
            }




app = Flask(__name__)
UPLOAD_DIR = "uploads"
os.makedirs(UPLOAD_DIR, exist_ok=True)

# logs interne
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('amerf_server.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# couleur
class ColoredFormatter(logging.Formatter):
    """Formateur de logs avec couleurs"""
    green = "\x1b[32m"
    yellow = "\x1b[33m"
    red = "\x1b[31m"
    reset = "\x1b[0m"

    def format(self, record):
        if record.levelno >= logging.ERROR:
            color = self.red
        elif record.levelno >= logging.WARNING:
            color = self.yellow
        else:
            color = self.green

        record.levelname = f"{color}{record.levelname}{self.reset}"
        return super().format(record)

# handler
for handler in logger.handlers:
    if isinstance(handler, logging.StreamHandler):
        handler.setFormatter(ColoredFormatter('%(asctime)s - %(levelname)s - %(message)s'))



class ProtectedYOLOManager:
    _instance = None
    _lock = threading.Lock()

    def __init__(self):
        self.model = None
        self.request_count = 0

    @classmethod
    def get_instance(cls):
        if cls._instance is None:
            with cls._lock:
                if cls._instance is None:
                    cls._instance = cls()
                    cls._instance._init_model()
        return cls._instance

    def _init_model(self):
        """Initialise le modèle YOLO protégé avec historique global"""
        try:
            logger.info("🚀 Initialisation du modèle YOLO protégé...")
            self.model = ProtectedYOLO(
                model_path="best.pt",
                watermark_key="yolo_protection_key",
                config={
                    'risk_threshold': 0.5,
                    'dpn_max_noise': 0.15,
                    'verbose': True
                }
            )
            logger.info(" Modèle YOLO protégé initialisé")

            # Afficher les stats initiales
            stats = self.model.get_protection_stats()
            global_stats = stats['global_stats']
            logger.info(f"📊 Historique chargé: {global_stats['total_queries']} requêtes, "
                       f"{global_stats['suspicious_queries']} suspectes "
                       f"({global_stats.get('suspicion_rate', 0):.1f}%)")

        except Exception as e:
            logger.error(f" Erreur d'initialisation du modèle: {e}")
            raise

# Initialisation du manager
manager = ProtectedYOLOManager.get_instance()


CLASS_NAMES = {
    0: 'aeroplane', 1: 'bicycle', 2: 'bird', 3: 'boat', 4: 'bottle',
    5: 'bus', 6: 'car', 7: 'cat', 8: 'chair', 9: 'cow',
    10: 'diningtable', 11: 'dog', 12: 'horse', 13: 'motorbike', 14: 'person',
    15: 'pottedplant', 16: 'sheep', 17: 'sofa', 18: 'train', 19: 'tvmonitor'
}

def results_to_dict(results):
    """Convertit les résultats YOLO en format simple pour le client"""
    if not results:
        return {"predictions": []}

    classes_detected = set()

    if isinstance(results, list):
        r = results[0]
    else:
        r = results

    if hasattr(r, 'boxes') and r.boxes is not None:
        for box in r.boxes:
            class_id = int(box.cls[0])
            class_name = CLASS_NAMES.get(class_id, "unknown")
            classes_detected.add(class_name)

    return {"predictions": list(classes_detected)}


@app.route("/predict", methods=["POST"])
def predict():
    """Endpoint unique pour les prédictions protégées - LOGS EN TEMPS RÉEL"""
    # Récupérer l'ID utilisateur
    user_id = request.headers.get("X-User-ID", "anonymous")

    # Incrémenter le compteur de requêtes
    manager.request_count += 1

    # Vérification de l'image
    if "image" not in request.files:
        logger.error(f" User:{user_id} - Aucune image fournie")
        return jsonify({"error": "No image provided"}), 400

    image_file = request.files["image"]
    filename = f"{uuid.uuid4()}.jpg"
    path = os.path.join(UPLOAD_DIR, filename)

    try:
        image_file.save(path)

        # LOG: Début de la requête
        logger.info(f" User:{user_id} - Début traitement (requête #{manager.request_count})")

        # Faire la prédiction avec le modèle protégé
        results, debug_info = manager.model.predict(
            source=path,
            user_id=user_id,
            conf=0.25,
            return_debug=True
        )

        # Récupérer les métriques de sécurité POUR LOGS
        fingerprint = debug_info['fingerprint']
        risk_score = fingerprint['risk_score']
        is_suspicious = fingerprint['is_suspicious']
        attack_type = fingerprint['attack_type']
        metrics = fingerprint['metrics']

        # Récupérer les métriques de précision
        real_precision = debug_info['real_precision']
        noisy_precision = debug_info['noisy_precision']
        noise_info = debug_info.get('noise_info')

        # Convertir les résultats pour le client
        response = results_to_dict(results)

        # LOG DÉTAILLÉ EN TEMPS RÉEL AVEC PRÉCISION
        if is_suspicious:
            logger.warning(f" User:{user_id} - COMPORTEMENT SUSPECT DÉTECTÉ")
            logger.warning(f"   Score risque: {risk_score:.3f} | Type: {attack_type.value}")
            logger.warning(f"   Métriques: Répétition={metrics['repeat_count']}x, "
                          f"Diversité={metrics['diversity_ratio']:.1%}, "
                          f"Fréquence={metrics['frequency_seconds']:.1f}s")
            logger.warning(f"   Précision AVANT bruit: {real_precision['avg_confidence']:.3f} (max: {real_precision['max_confidence']:.3f})")
            logger.warning(f"   Précision APRÈS bruit: {noisy_precision['avg_confidence']:.3f} (max: {noisy_precision['max_confidence']:.3f})")
            if noise_info:
                logger.warning(f"   Δ Confiance: {noise_info['avg_confidence_change']:.3f} (max Δ: {noise_info['max_confidence_change']:.3f})")
                logger.warning(f"   Niveau bruit: {noise_info['noise_level']:.3f}")
            logger.warning(f"   Hash image: {debug_info['image_hash']}")
        else:
            logger.info(f" User:{user_id} - Comportement normal")
            logger.info(f"   Score risque: {risk_score:.3f} | "
                       f"Détections: {len(response['predictions'])} objets")
            logger.info(f"   Précision: {real_precision['avg_confidence']:.3f} (max: {real_precision['max_confidence']:.3f})")

        # Optionnel: ajouter les métriques si header X-Debug est présent
        if request.headers.get("X-Debug", "").lower() == "true":
            response["security"] = {
                "risk_score": round(risk_score, 3),
                "is_suspicious": is_suspicious,
                "attack_type": attack_type.value,
                "metrics": metrics,
                "precision": {
                    "real": real_precision,
                    "noisy": noisy_precision if is_suspicious else real_precision
                }
            }

        # Nettoyer le fichier temporaire
        os.remove(path)

        # LOG: Fin de la requête
        logger.info(f" User:{user_id} - Prédiction envoyée ({len(response['predictions'])} objets)")

        return jsonify(response)

    except Exception as e:
        if os.path.exists(path):
            os.remove(path)
        logger.error(f"❌ User:{user_id} - Erreur: {str(e)}")
        return jsonify({"error": str(e)}), 500


@app.route("/stats", methods=["GET"])
def get_stats():
    """Endpoint pour obtenir les statistiques"""
    user_id = request.args.get("user_id")

    logger.info(f"📊 Demande de statistiques pour user: {user_id or 'global'}")

    if user_id:
        stats = manager.model.get_protection_stats(user_id)
        return jsonify({
            'user_stats': stats.get('user_stats', {}),
            'global_summary': stats.get('global_stats', {})
        })
    else:
        stats = manager.model.get_protection_stats()
        return jsonify({
            'global_stats': stats['global_stats'],
            'model_stats': stats.get('model_stats', {}),
            'server_requests': manager.request_count
        })

 Chargement du modèle YOLO depuis best.pt...
✅ Historique chargé: 5 requêtes
Modèle YOLO protégé 


In [ ]:

if __name__ == "__main__":
    # Vérifier que le modèle est chargé
    try:
        _ = manager.model
    except Exception as e:
        logger.error(f"❌ Le modèle n'a pas pu être chargé: {e}")
        exit(1)

    # Configurer ngrok
    try:
        print("\n" + "="*60)
        print("🌐 Configuration de ngrok...")



        # Ouvrir le tunnel
        public_url = ngrok.connect(5000).public_url
        print(f" Ngrok configuré: {public_url}")

    except Exception as e:
        print(f"  Ngrok non disponible: {e}")
        public_url = "http://localhost:5000"

    print("\n" + "="*60)
    print(" SERVEUR AMERF-YOLO PRÊT - LOGS EN TEMPS RÉEL")
    print("="*60)
    print(f" URL publique: {public_url}")

    print(f" Endpoint stats: GET {public_url}/stats?user_id=USER_ID")
    print("="*60)
    print(" Logs temps réel:")
    print("   Début traitement")
    print("  Comportement normal")
    print("   COMPORTEMENT SUSPECT (détails affichés)")
    print("  Résultats envoyés")
    print("   Erreurs")
    print("="*60)
    print(" Historique: amerf_global_history.json")
    print("="*60 + "\n")

    # Fonction pour afficher un résumé périodique
    def log_summary():
        try:
            stats = manager.model.get_protection_stats()
            global_stats = stats['global_stats']
            logger.info(f"\n RÉSUMÉ GLOBAL ({datetime.now().strftime('%H:%M:%S')})")
            logger.info(f"   Requêtes totales: {global_stats['total_queries']}")
            logger.info(f"   Requêtes suspectes: {global_stats['suspicious_queries']}")
            logger.info(f"   Taux suspicion: {global_stats.get('suspicion_rate', 0):.1f}%")
            logger.info(f"   Utilisateurs uniques: {global_stats['unique_users']}")

            # Afficher la distribution des attaques
            attack_dist = global_stats.get('attack_distribution', {})
            if attack_dist:
                logger.info("   Distribution attaques:")
                for attack_type, count in attack_dist.items():
                    logger.info(f"     • {attack_type}: {count}")

        except Exception as e:
            logger.error(f"Erreur dans log_summary: {e}")

    # Démarrer le thread de logging périodique
    def periodic_summary():
        while True:
            time.sleep(300)  # Toutes les 5 minutes
            log_summary()

    summary_thread = threading.Thread(target=periodic_summary, daemon=True)
    summary_thread.start()

    # Enregistrer le cleanup
    def cleanup():
        print("\n🛑 Arrêt du serveur...")
        try:
            ngrok.kill()  # Fermer le tunnel ngrok
        except:
            pass
        manager.model._cleanup()  # Sauvegarder l'historique
        logger.info("👋 Serveur arrêté proprement")
        print(" Historique sauvegardé, ngrok fermé")

    atexit.register(cleanup)

    # Démarrer Flask
    print(" Démarrage du serveur Flask...")
    logger.info(f"Serveur démarré sur {public_url}")

    try:
        app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)
    except KeyboardInterrupt:
        print("\n\n⏹  Arrêt demandé par l'utilisateur")
        cleanup()


🌐 Configuration de ngrok...
✅ Ngrok configuré: https://nonantagonistic-ileana-nonradiating.ngrok-free.dev

🚀 SERVEUR AMERF-YOLO PRÊT - LOGS EN TEMPS RÉEL
🌍 URL publique: https://nonantagonistic-ileana-nonradiating.ngrok-free.dev
 Endpoint stats: GET https://nonantagonistic-ileana-nonradiating.ngrok-free.dev/stats?user_id=USER_ID
 Logs temps réel:
   Début traitement
  Comportement normal
   COMPORTEMENT SUSPECT (détails affichés)
  Résultats envoyés
   Erreurs
 Historique: amerf_global_history.json

 Démarrage du serveur Flask...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit



0: 640x448 1 train, 1812.5ms
Speed: 26.1ms preprocess, 1812.5ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 448)


INFO:werkzeug:127.0.0.1 - - [24/Dec/2025 22:12:50] "POST /predict HTTP/1.1" 200 -



 User: Sara
 Score risque: 0.422 | Type: normal
Métriques:
  • Fréquence: 178.7s (score: 0.03)
  • Diversité: 33.3% (score: 0.67)
  • Répétition: 1x (score: 0.30)
  • Images uniques: 2/6
📊 PRÉCISION DES CONFIDENCES:
  • Réelle (avant bruit):
     - Moyenne: 0.884
     - Max: 0.884
     - Min: 0.884
     - Détections: 1
  • Après bruit: AUCUN (utilisateur normal)
 Défenses: Bruit appliqué: False
 Performance: 2.52s

0: 640x448 1 train, 1063.5ms
Speed: 6.5ms preprocess, 1063.5ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 448)


INFO:werkzeug:127.0.0.1 - - [24/Dec/2025 22:15:45] "POST /predict HTTP/1.1" 200 -



 User: Sara
 Score risque: 0.888 | Type: cloning
Métriques:
  • Fréquence: 152.9s (score: 0.03)
  • Diversité: 25.0% (score: 0.75)
  • Répétition: 3x (score: 0.90)
  • Images uniques: 2/8
📊 PRÉCISION DES CONFIDENCES:
  • Réelle (avant bruit):
     - Moyenne: 0.884
     - Max: 0.884
     - Min: 0.884
     - Détections: 1
  • Après bruit (niveau: 0.129):
     - Moyenne: 0.894
     - Max: 0.894
     - Min: 0.894
     - Δ Moyenne: 0.009
     - Δ Max: 0.009
     - Boîtes affectées: 1
 Défenses: Bruit appliqué: True
 Performance: 1.08s

0: 640x448 1 train, 1164.4ms
Speed: 6.4ms preprocess, 1164.4ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 448)


INFO:werkzeug:127.0.0.1 - - [24/Dec/2025 22:22:59] "POST /predict HTTP/1.1" 200 -



 User: Sara
 Score risque: 0.929 | Type: cloning
Métriques:
  • Fréquence: 167.1s (score: 0.03)
  • Diversité: 20.0% (score: 0.80)
  • Répétition: 5x (score: 0.90)
  • Images uniques: 2/10
📊 PRÉCISION DES CONFIDENCES:
  • Réelle (avant bruit):
     - Moyenne: 0.884
     - Max: 0.884
     - Min: 0.884
     - Détections: 1
  • Après bruit (niveau: 0.136):
     - Moyenne: 0.883
     - Max: 0.883
     - Min: 0.883
     - Δ Moyenne: 0.001
     - Δ Max: 0.001
     - Boîtes affectées: 1
 Défenses: Bruit appliqué: True
 Performance: 1.19s
